In [22]:
from tqdm import tqdm

def create_example(all_data):
    all_result = []
    for data in tqdm(all_data):
        data_id = data["_id"]
        Question = data["question"]
        answer = data["answer"]
        context = data["context"]
        supporting_facts = data["supporting_facts"]
        support_dic = {}
        for sup_sent in supporting_facts:
            title = sup_sent[0]  # supporting fact의 제목
            set_num = sup_sent[1]  # 문장번호
            if title not in support_dic.keys():
                support_dic[title] = []
            support_dic[title].append(set_num)
        
        supporting_num_list = []
        sent_list = []
        sentences = ""
        sent_num = 1
        for index, j in enumerate(context):
            title = j[0]
            sent_list.append(title.strip()+ ".")
            sent_num += 1
            
            if title in support_dic.keys():
                for i in support_dic[title]:
                    supporting_num_list.append(sent_num+ i )
    
            if sentences == "":
                sentences = title + ". "
            else:
                sentences = sentences + " " + title + ". "
            for sent in j[1]:
                sentences = sentences + sent
                sent_list.append(sent.strip())
                sent_num += 1
            
            assert (sent_num -1) == len(sent_list)
            
        result = {}
        result["_id"] = data_id
        result["question"] = Question
        result["document"] = sentences
        result["sent"] = sent_list
        result["supporting_num"] = supporting_num_list
        result["output"] = answer
        

        all_result.append(result)

    return all_result
    

In [23]:
import json
# file_path = "../data/origin/hotpot_train_v1.1_re.json"
file_path = "../data/origin/hotpot_dev.json"
with open(file_path, "r", encoding="utf-8") as file:
    dev_data = json.load(file)

input_data = create_example(dev_data)

100%|██████████| 7405/7405 [00:00<00:00, 22717.28it/s]


In [24]:
with open("../data/1029data/hotpot_dev_supporting.json", "w", encoding="utf-8") as f:
    json.dump(input_data, f, ensure_ascii=False, indent=4)

In [4]:
from tqdm import tqdm
all_len = []
all_result = []

for input_data_ in tqdm(input_data):
    messages = [
        {"role": "system", "content": input_data_["question"]}, 
        {"role": "user", "content": input_data_["document"]},
        {"role":"assistant", "content": input_data_["output"]},
    ]
    count = len(tokenizer.apply_chat_template(messages))
    if count <= 2044:
        all_result.append(input_data_)
    all_len.append(count)

100%|██████████| 90447/90447 [03:55<00:00, 383.31it/s]


In [5]:
print(len(input_data))
print(len(all_result))

90447
86434


In [6]:
with open("../data/1008data/hotpot_train.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [7]:
from tqdm import tqdm
all_len = []
all_result = []
over_num = 0
for input_data_ in tqdm(input_data):
    text = input_data_["all_text"]
    count = len(tokenizer(text)["input_ids"])
    if count <= 2048:
        all_result.append(input_data_)
    else:
        over_len = count - 2048
        input_data_['text'] = input_data_['text'][:over_len]
        over_num +=1
    # all_len.append(len(tokenizer(text)["input_ids"]))
print(over_num)

100%|██████████| 7405/7405 [00:18<00:00, 405.87it/s]

668


In [8]:
for input_data_ in tqdm(input_data):
    del input_data_["all_text"]

100%|██████████| 7405/7405 [00:00<00:00, 1753249.85it/s]


In [10]:
with open("../data/qwen_hotpot_test_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [37]:
from tqdm import tqdm
all_len = []
all_result = []

for input_data_ in tqdm(input_data):
    text = input_data_["text"]
    if len(tokenizer(text)["input_ids"]) <= 2048:
        # data["text"] = data["text"]
        all_result.append(input_data_)
    # all_len.append(len(tokenizer(text)["input_ids"]))

100%|██████████| 90447/90447 [04:12<00:00, 357.70it/s]


In [38]:
with open("data/qwen_hotpot_train_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [16]:
with open("data/teddst_dev.json", "w", encoding="utf-8") as f:
    json.dump(input_data, f, ensure_ascii=False, indent=4)

In [7]:
print(input_data[0]["label"])

**Answer**: yes
**Supporting Sentences**: [4] Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.
[17] Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.



In [16]:
count = len(list(filter(lambda x: x < 2048, all_len)))
print(count)

6539
